In [2]:
# Criar resumos para cada uma das fases do Jogo em que se tenha uma visualização básica 
# do que aconteceu no jogo
# Criar um grafo mostrando a probabilidade condicional de algo acontecer dado o que aconteceu no jogo
# Tentar fazer a adição de dados utilizando a voz
# Adicionar novas possibilidades de levantamento, aberto fechado.

import pandas as pd
df = pd.read_csv("2025-11-17T00-08_export.csv")
df_passe = df[df['Acao'] == 'Passe']
total_recepcoes = len(df_passe)
df_passe_agg = df_passe.groupby(['Jogador']).agg(total_recepcoes=("Jogador", "count")).reset_index()
df_passe_agg['total_recepcoes_jogo'] = total_recepcoes
df_passe_agg['razao_recepcoes'] = df_passe_agg['total_recepcoes']/df_passe_agg['total_recepcoes_jogo']*100
if len(df_passe_agg) == 1:
    jogador_com_recepcao = df_passe_agg.loc[0, 'Jogador']
    jogador_sem_recepcao = "player2" if jogador_com_recepcao == "player1" else "player1"   
    df_passe_agg.loc[1,::] = [jogador_sem_recepcao, 0, 0, 0]

In [5]:
total_pontos_player1 = df[(df['Jogador'] == 'player1') & (df['Qualidade'] == 'Ponto')]
total_recepcoes_player_1 = df[(df['Jogador'] == 'player1') & (df['Acao'] == 'Passe')]
df[(df['Jogador'] == 'player2')] # & (df['Acao'] == 'Passe')]

,Unnamed: 0,Complexo,Etapa,Acao,Qualidade,Jogador,Lance,Ponto
1,0,K1,Fase2,Levantamento,A,player2,1,0
3,0,K1,Fase1,Passe,A,player2,0,1
5,0,K1,Fase3,Ataque no meio,Erro,player2,2,1
6,0,K1,Fase1,Passe,A,player2,0,2
8,0,K1,Fase3,Ataque no meio,A,player2,2,2
10,0,K1,Fase2,Levantamento,A,player2,1,3


In [4]:
import plotly.graph_objects as go

def donut_recepcoes(df):
    """
    Cria um donut chart a partir de um DataFrame com:
    - df['player']
    - df['total_recepcoes']
    Cada linha é um jogador.
    Return:
        fig (plotly.graph_objs._figure.Figure)
    """

    # Valores
    players = df["Jogador"].tolist()
    recepcoes = df["total_recepcoes"].tolist()
    # Total
    total = sum(recepcoes)
    # Porcentagens
    porcentagens = [(r / total) * 100 for r in recepcoes]
    # Donut chart
    fig = go.Figure(
        data=[
            go.Pie(
                labels=players,
                values=recepcoes,
                hole=0.55,
                textinfo="label+percent",
                hovertemplate="<b>%{label}</b><br>Recepções: %{value}<extra></extra>",
                marker=dict(
                    colors=["#2563eb", "#10b981", "#f59e0b", "#ef4444"]  # suporta + jogadores
                )
            )
        ]
    )
    # Layout clean
    fig.update_layout(
        title_text="Receptions by Player",
        margin=dict(l=20, r=20, t=40, b=20),
        height=350,
        showlegend=True
    )

    return fig
fig = donut_recepcoes(df_passe_agg)
fig.show()

# Grafo

In [1]:
import plotly.graph_objects as go
import networkx as nx

# Dicionário de ações e probabilidades de transição
acoes = {
    'A': {'B': 0.5, 'C': 0.5},
    'B': {'A': 0.3, 'D': 0.7},
    'C': {'A': 0.4, 'F': 0.6},
    'D': {'B': 0.2, 'E': 0.8},
    'E': {'D': 0.1, 'F': 0.9},
    'F': {'C': 0.3, 'E': 0.7}
}

# Creat um grafo direcionado
G = nx.DiGraph()

# Adicionar ações ao grafo
for acao, transicoes in acoes.items():
    G.add_node(acao)
    for proxima_acao, probabilidade in transicoes.items():
        G.add_edge(acao, proxima_acao, weight=probabilidade)

# Posições dos nós
pos = nx.spring_layout(G)

# Criar listas para armazenar as coordenadas dos nós e arestas
node_x = []
node_y = []
edge_x = []
edge_y = []
edge_weights = []

# Adicionar coordenadas dos nós
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

# Adicionar coordenadas das arestas
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    edge_weights.append(G[edge[0]][edge[1]]['weight'])

# Criar o grafo
fig = go.Figure(data=[go.Scatter(
    x=edge_x,
    y=edge_y,
    mode='lines',
    line=dict(width=edge_weights, color='gray'),
    hoverinfo='none'
),
go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    hoverinfo='text',
    hovertext=[f'Node: {node}' for node in G.nodes()],
    marker=dict(size=10)
)])

# Adicionar arestas ao grafo
for i, edge in enumerate(G.edges()):
    fig.add_annotation(
        x=(pos[edge[0]][0] + pos[edge[1]][0]) / 2,
        y=(pos[edge[0]][1] + pos[edge[1]][1]) / 2,
        text=str(G[edge[0]][edge[1]]['weight']),
        showarrow=False
    )

# Exibir o grafo
fig.update_layout(
    title='Grafo de Ações',
    showlegend=False,
    hovermode='x',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)
fig.show()

ModuleNotFoundError: No module named 'networkx'